In [10]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
print(features.shape)
print(targets.shape)

(360, 6)
(360,)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [53]:
import tensorflow as tf
import numpy as np

n = features.shape[0]

X = tf.placeholder(tf.float32, (None,6))
Y_real = tf.placeholder(tf.float32, (None,))

W1 = tf.Variable(tf.zeros([6, 2])) # initialize with 0s first
b1 = tf.Variable(tf.zeros(2))
W2 = tf.Variable(tf.zeros([2, 1])) # initialize with 0s first
b2 = tf.Variable(tf.zeros(1))

Z1 = tf.matmul(X,W1) + b1
A1 = tf.nn.sigmoid(Z1)
Z2 = tf.matmul(A1,W2) + b2
output = tf.nn.sigmoid(Z2)



loss = tf.reduce_mean((output-Y_real)**2)


predictions = tf.cast(output > 0.5, tf.int32)
y = tf.cast(Y_real, tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, y), tf.float32))

train_op=tf.train.AdamOptimizer(0.01).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

TRAIN_STEPS = 100
for i in range(TRAIN_STEPS+1):
    sess.run(train_op, feed_dict={X: features, Y_real: targets})
    if i % 20 == 0:
        print('Accuracy = ',sess.run(accuracy, feed_dict={X: features_test, Y_real: targets_test}))
        print('Loss = ', sess.run(loss, feed_dict={X: features, Y_real: targets}))
sess.close()

Accuracy =  0.65
Loss =  0.2482006
Accuracy =  0.65
Loss =  0.22144744
Accuracy =  0.65
Loss =  0.21537918
Accuracy =  0.65
Loss =  0.2154595
Accuracy =  0.65
Loss =  0.21537326
Accuracy =  0.65
Loss =  0.21537843
